In [6]:
import os
import pandas as pd
from collections import Counter
from kiwipiepy import Kiwi
import json

# 데이터셋 경로 설정
data_folder = "../Completed_csv"

# 형태소 분석기 초기화
kiwi = Kiwi()

# 불용어 리스트
custom_stopwords = [
    "지난해", "이날", "오늘", "내일", "올해", "시간", "관련", 
    "있다", "없다", "위해", "대한", "그리고", "등", "수", "것"
]

# 명사 추출
def extract_nouns(text):
    results = []
    analysis = kiwi.analyze(text)
    for token, pos, _, _ in analysis[0][0]:
        if len(token) > 1 and (pos.startswith('N') or pos.startswith('SL')) and token not in custom_stopwords:
            results.append(token)
    return results

# CSV 파일 로드 및 텍스트 결합
def load_and_combine_text(data_folder):
    combined_texts = []
    all_data = pd.DataFrame()

    for filename in os.listdir(data_folder):
        if filename.endswith(".csv"):
            filepath = os.path.join(data_folder, filename)
            df = pd.read_csv(filepath, encoding="utf-8")
            if "title" in df.columns and "short_content" in df.columns:
                df["combined"] = df["title"].fillna("") + " " + df["short_content"].fillna("")
                combined_texts.extend(df["combined"].tolist())
                all_data = pd.concat([all_data, df], ignore_index=True)

    return combined_texts, all_data

# 키워드 추출
def extract_top_keywords(texts, top_n=50):
    all_nouns = []
    for text in texts:
        nouns = extract_nouns(text)
        all_nouns.extend(nouns)
    word_counts = Counter(all_nouns)
    return word_counts.most_common(top_n)

# JSON 파일로 데이터 저장
def save_preview_to_json(df, keywords, output_file="preview.json"):
    preview_data = {
        "keywords": [kw for kw, _ in keywords],
        "articles": {
            keyword: df[df["combined"].str.contains(keyword)].to_dict(orient="records")
            for keyword, _ in keywords
        },
        "company_stats": dict(Counter(df["company"])),
        "keyword_stats": {
            keyword: len(df[df["combined"].str.contains(keyword)]) for keyword, _ in keywords
        },
    }

    # JSON 파일 저장
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(preview_data, f, ensure_ascii=False, indent=4)

    print(f"\nPreview data saved to {output_file}.")

def save_to_redis(df, keywords):
    print("Starting to save data to Redis...")

    # 키워드 목록 저장
    r.sadd("keywords", *[kw for kw, _ in keywords])
    print("Keywords saved.")

    # 키워드별 기사 저장
    for keyword, _ in keywords:
        print(f"Saving articles for keyword: {keyword}")
        articles = df[df["combined"].str.contains(keyword)]
        for _, row in articles.iterrows():
            r.rpush(f"keyword:{keyword}:articles", json.dumps(row.to_dict()))
        print(f"Saved {len(articles)} articles for keyword: {keyword}.")

    # 언론사별 기사 개수 저장
    print("Saving company stats...")
    company_counts = Counter(df["company"])
    for company, count in company_counts.items():
        r.hset("company:stats", company, count)
    print("Company stats saved.")

    # 키워드별 기사 개수 저장
    print("Saving keyword stats...")
    for keyword, _ in keywords:
        count = len(df[df["combined"].str.contains(keyword)])
        r.hset("keyword:stats", keyword, count)
        print(f"Keyword '{keyword}' has {count} articles.")
    print("Keyword stats saved.")

    print("Data saved to Redis.")

# 실행
if __name__ == "__main__":
    from dotenv import load_dotenv
    load_dotenv()

    # 데이터 로드 및 텍스트 결합
    combined_texts, df = load_and_combine_text(data_folder)

    # 상위 50개 키워드 추출
    top_keywords = extract_top_keywords(combined_texts, top_n=50)
    
    # 상위 키워드 출력
    print("\n상위 50개 키워드:")
    for rank, (word, count) in enumerate(top_keywords, start=1):
        print(f"{rank}. {word} - {count}회 등장")

    # Redis에 저장
    save_to_redis(df, top_keywords)


상위 50개 키워드:
1. 대통령 - 868회 등장
2. 윤석열 - 357회 등장
3. 체포 - 321회 등장
4. 영장 - 281회 등장
5. 서울 - 274회 등장
6. 구속 - 211회 등장
7. 내란 - 191회 등장
8. 미국 - 189회 등장
9. 공수처 - 186회 등장
10. 국민 - 182회 등장
11. 조사 - 178회 등장
12. 정부 - 172회 등장
13. 집행 - 160회 등장
14. 경찰 - 159회 등장
15. 혐의 - 149회 등장
16. 트럼프 - 136회 등장
17. 심사 - 135회 등장
18. 한국 - 130회 등장
19. 국회 - 126회 등장
20. 장관 - 125회 등장
21. 오후 - 125회 등장
22. 대행 - 118회 등장
23. 비상계엄 - 116회 등장
24. 대표 - 107회 등장
25. 지원 - 107회 등장
26. 탄핵 - 106회 등장
27. 권한 - 105회 등장
28. 시장 - 102회 등장
29. 수사 - 100회 등장
30. 국가 - 95회 등장
31. 의원 - 93회 등장
32. 범죄 - 93회 등장
33. 서부 - 91회 등장
34. 경호처 - 91회 등장
35. 기업 - 90회 등장
36. 진행 - 89회 등장
37. 계엄 - 89회 등장
38. 경제 - 89회 등장
39. 출석 - 88회 등장
40. 관저 - 87회 등장
41. 지법 - 84회 등장
42. 특검법 - 82회 등장
43. 경기 - 82회 등장
44. 오전 - 80회 등장
45. 이후 - 80회 등장
46. 시작 - 79회 등장
47. 최상목 - 79회 등장
48. 행사 - 77회 등장
49. 실질 - 77회 등장
50. 사태 - 77회 등장
Starting to save data to Redis...
Keywords saved.
Saving articles for keyword: 대통령
Saved 437 articles for keyword: 대통령.
Saving articles for keyword: 윤석열
Saved